In [1]:
import pandas as pd

In [2]:
csv = pd.read_csv('docs/Phenotypic_V1_0b_preprocessed1.csv')

In [3]:
csv.head()

,Unnamed: 0,Unnamed: 0.1,SUB_ID,X,subject,SITE_ID,FILE_ID,DX_GROUP,DSM_IV_TR,AGE_AT_SCAN,...,qc_notes_rater_1,qc_anat_rater_2,qc_anat_notes_rater_2,qc_func_rater_2,qc_func_notes_rater_2,qc_anat_rater_3,qc_anat_notes_rater_3,qc_func_rater_3,qc_func_notes_rater_3,SUB_IN_SMP
0,0,1,50002,1,50002,PITT,no_filename,1,1,16.77,...,NaN,OK,NaN,fail,ic-parietal-cerebellum,OK,NaN,fail,ERROR #24,1
1,1,2,50003,2,50003,PITT,Pitt_0050003,1,1,24.45,...,NaN,OK,NaN,OK,NaN,OK,NaN,OK,NaN,1
2,2,3,50004,3,50004,PITT,Pitt_0050004,1,1,19.09,...,NaN,OK,NaN,OK,NaN,OK,NaN,OK,NaN,1
3,3,4,50005,4,50005,PITT,Pitt_0050005,1,1,13.73,...,NaN,OK,NaN,maybe,ic-parietal-cerebellum,OK,NaN,OK,NaN,0
4,4,5,50006,5,50006,PITT,Pitt_0050006,1,1,13.37,...,NaN,OK,NaN,maybe,ic-parietal slight,OK,NaN,OK,NaN,1


In [5]:
csv.AGE_AT_SCAN.head()

0    16.77
1    24.45
2    19.09
3    13.73
4    13.37
Name: AGE_AT_SCAN, dtype: float64

In [6]:
csv.AGE_AT_MPRAGE.head()

0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
Name: AGE_AT_MPRAGE, dtype: float64

## Set keys (column names) that we want to split equally
## and number of iterations

In [85]:
keys = ['SITE_ID', 'SEX']
num_iter = 3

In [8]:
csv[keys].head()

,SITE_ID,SEX
0,PITT,1
1,PITT,1
2,PITT,1
3,PITT,2
4,PITT,1


## make labels 

In [82]:
labels = [ '.'.join([str(csv[key][i]) for key in keys]) for i in range(len(csv))]
print labels[:5]

['PITT.1', 'PITT.1', 'PITT.1', 'PITT.2', 'PITT.1']


## split

In [9]:
from sklearn.cross_validation import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(labels, num_iter, test_size=0.5, random_state=0)

In [80]:
print(sss)

StratifiedShuffleSplit(labels=['PITT.1' 'PITT.1' 'PITT.1' ..., 'SBL.1' 'MAX_MUN.2' 'MAX_MUN.1'], n_iter=3, test_size=0.5, random_state=0)


In [88]:
for index_a, index_b in sss:
    #print("TRAIN:", train_index, "TEST:", test_index)
    set_a, set_b = csv.iloc[index_a], csv.iloc[index_b]
    print 'iteration'
    print 'sex  == 1:', len(set_a[set_a.SEX == 1]), len(set_b[set_b.SEX == 1])
    print 'site == yale:', len(set_a[set_a.SITE_ID == 'YALE']), len(set_b[set_b.SITE_ID == 'YALE'])


iteration
sex  == 1: 474 474
site == yale: 28 28
iteration
sex  == 1: 473 475
site == yale: 28 28
iteration
sex  == 1: 473 475
site == yale: 28 28


In [11]:
from sklearn.cross_validation import StratifiedShuffleSplit

In [22]:
df = pd.read_csv('docs/Phenotypic_V1_0b_preprocessed1.csv')

In [23]:
def split_data(df, keys, num_iter=3, random_state=0):
    labels = [ '.'.join([str(df[key].iloc[i]) for key in keys]) for i in range(len(df))]
    return StratifiedShuffleSplit(labels, num_iter, test_size=0.5, random_state=random_state)


In [17]:
keys

['AGE_AT_SCAN', 'SITE_ID', 'SEX']

In [28]:
df.columns

Index([u'Unnamed: 0', u'Unnamed: 0.1', u'SUB_ID', u'X', u'subject', u'SITE_ID',
       u'FILE_ID', u'DX_GROUP', u'DSM_IV_TR', u'AGE_AT_SCAN',
       ...
       u'qc_anat_rater_2', u'qc_anat_notes_rater_2', u'qc_func_rater_2',
       u'qc_func_notes_rater_2', u'qc_anat_rater_3', u'qc_anat_notes_rater_3',
       u'qc_func_rater_3', u'qc_func_notes_rater_3', u'SUB_IN_SMP',
       u'AGE_AT_SCAN_binary'],
      dtype='object', length=107)

In [41]:
keys = ['AGE_AT_SCAN', 'SEX' ,'SITE_ID']

# age -> 0:young/1:old (median)
for i, key in enumerate(keys):
    if 'AGE' in key:    # TODO: better way?
    #if len(df[key].unique())>2:
        key_binary = '%s_binary' % key
        df[key_binary] = pd.qcut(df[key], 2, labels=[0,1])
        keys[i] = key_binary

sss = split_data(df, keys, num_iter=3, random_state=0)
print sss
assess = []
for index_a, index_b in sss:
    #print("TRAIN:", train_index, "TEST:", test_index)
    set_a, set_b = csv.iloc[index_a], csv.iloc[index_b]
    # image data[index_a/b]

    # run training

ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of labels for any class cannot be less than 2.

In [20]:
for site in df['SITE_ID'].unique():
    print '%s: %s' % (site, len(df[df.SITE_ID == site]))

PITT: 57
OLIN: 36
OHSU: 28
SDSU: 36
TRINITY: 49
UM_1: 110
UM_2: 35
USM: 101
YALE: 56
CMU: 27
LEUVEN_1: 29
LEUVEN_2: 35
KKI: 55
NYU: 184
STANFORD: 40
UCLA_1: 82
UCLA_2: 27
MAX_MUN: 57
CALTECH: 38
SBL: 30


In [21]:
df[df.SITE_ID == 'CMU'].SEX

497    1
498    1
499    2
500    1
501    1
502    1
503    2
504    1
505    2
506    1
507    1
508    1
509    1
510    1
511    2
512    1
513    1
514    1
515    1
516    2
517    1
518    1
519    1
520    1
521    1
522    1
523    2
Name: SEX, dtype: int64

In [ ]:
labels = [ '.'.join([str(df[key].iloc[i]) for key in keys]) for i in range(len(df))]


In [ ]:
import numpy as np

In [ ]:
rows, index, count = np.unique(labels, return_index=True, return_counts=True)

In [47]:
index[ count == 1 ]

array([ 82, 326, 105, 579, 154, 907, 320])

In [48]:
df_sub = df.drop(df.index[index[count==1].tolist()], inplace=False)

In [64]:
labels_sub = [ '.'.join([str(df_sub[key].iloc[i]) for key in keys]) for i in range(len(df_sub))]


In [65]:
key, index, count = np.unique(labels_sub, return_index=True, return_counts=True)

In [66]:
count

array([42, 16, 13, 88, 27,  9, 15, 12, 32, 15, 51, 23, 58, 11, 11, 31, 13,
        7, 22,  6,  6,  8, 10,  2, 15, 12, 30, 21, 29, 11, 37, 59, 22, 34,
       30, 17, 34, 20,  2, 26, 22, 90,  9,  8,  6,  7, 15,  4,  2, 11,  4])

In [63]:
df_sub['SITE_ID'].iloc[0]

'PITT'

In [56]:
len(df)

1112